In [8]:
import os
import pandas as pd
from langchain.llms.openai import OpenAI
import traceback

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv() #Take environment variables from .env 
KEY=os.getenv("OPENAI_API_KEY")
llm=ChatOpenAI(openai_api_key=KEY, model_name = "gpt-3.5-turbo", temperature=0.5)

/tmp/ipykernel_92431/341901509.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(openai_api_key=KEY, model_name = "gpt-3.5-turbo", temperature=0.5)


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)